In [ ]:
from requests import get
from bs4 import BeautifulSoup

In [ ]:
base_rebec_url = 'http://www.ensaiosclinicos.gov.br/rg/?page='
download_path = '/Users/nicholasdevito/Desktop/Test Rebec'

In [ ]:
#Quick way to get max_page count
url = base_rebec_url + '1'
response = get(url, verify = False)
html = response.content

#gets parsed XML
soup = BeautifulSoup(html, "html.parser")

number_of_pages = soup.find('div', {'class': 'pagination'})
max_page = number_of_pages.find_all('a')[-3].get_text()
print(max_page)

In [ ]:
#everything we need to run our selenium crawler
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
#from selenium.common.exceptions import TimeoutException
from time import sleep

chrome_driver = '/Users/nicholasdevito/Dropbox/Python projects/ANZCTR/chromedriver'
page_list = [str(i) for i in range(2,5)]#after testing put this for full download: 'int(max_page)+1' instead of '5'

In [ ]:
#this goes to the first search page, downloads the xml, then does that for every other page

chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : download_path,
        'profile.managed_default_content_settings.images':2,
        'disk-cache-size': 4096}
chromeOptions.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(executable_path=chrome_driver, options=chromeOptions)
driver.get(base_rebec_url + '1')
#sleep(3)
driver.find_element_by_id("chk_toggle_selection").click()
driver.find_element_by_xpath("//input[@value='Download selected as OpenTrials XML format']").click()
#sleep(3)
for page in page_list:
    driver.find_element_by_link_text(page).click()
    driver.find_element_by_id("chk_toggle_selection").click()
    driver.find_element_by_xpath("//input[@value='Download selected as OpenTrials XML format']").click()
    #sleep(1)
sleep(5)
driver.quit()

 

In [ ]:
#quick check to make sure we got the correct number of files

dl_counter = 0
for file in os.listdir(download_path):  
    if file.endswith('.xml'):
        dl_counter += 1

if dl_counter == 4:  #4 is here just for testing, use 'max_page' variable for full implementation
    print("Full Download Successful")
else:
    print("Missing " + str(4 - dl_counter) + " File(s)") #once again, replace the 4 here with 'max_page' in full implementation

In [ ]:
import xmltodict
import json
import os

In [ ]:
#gets each trial and adds it to a list a json

rebec_trials = []

for file in os.listdir(download_path):
    if file.endswith('.xml'):
        with open(download_path + '/' + file) as xml:
            trials = xmltodict.parse(xml.read())
            for trial in trials['trials']['trial']:
                rebec_trials.append(str(json.dumps(trial)))

In [ ]:
#quick checks
len(rebec_trials)

In [ ]:
#and another
print(rebec_trials[5])

In [ ]:
#creates a csv with no headers and a single column with all trials in json in their own row.

import csv
with open('test.csv','w', newline = '')as test_csv:
    writer=csv.writer(test_csv)
    for val in rebec_trials:
        writer.writerow([val])